In [60]:
import time
import os
import re
from wordcloud import WordCloud,STOPWORDS
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import seaborn as sns
os.chdir(r'c:/users/vasil/desktop/parsing_data')
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords
import pymorphy2
morph = pymorphy2.MorphAnalyzer()


In [61]:

def clear(txt):
    tokens=word_tokenize(txt)
    tokens=[morph.parse(w)[0].normal_form for w in tokens]
    tbl=str.maketrans('', '',string.punctuation)
    stripped=[w.translate(tbl) for w in tokens]
    stop_words = stopwords.words('russian')                                                               
    words=[word for word in stripped if word.isalpha() and word not in stop_words] 
    return str(words)

#vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3), norm='l2')

data = pd.read_csv('clean_data_1.csv', sep=';', encoding='windows-1251')
data = data.drop('Unnamed: 0', axis=1)
labels = data.loc[:,'salary':'court_threat'].astype('category')
data['text'] = data['text'].map(lambda x: clear(str(x)))
num_labels = 7




In [62]:
#train, test = train_test_split(data['text'], labels, random_state=42, test_size=0.20, shuffle=True)
train, test = train_test_split(data,random_state=42, test_size=0.2, shuffle=True)

In [63]:

X_train = train.text
X_test = test.text

In [64]:
# using binary relevance
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV

#https://towardsdatascience.com/multi-class-text-classification-model-comparison-and-selection-5eb066197568

SVC_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stopwords.words('russian'))),
                ('clf', BinaryRelevance(LinearSVC())),
            ])

NB_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stopwords.words('russian'))),
                ('clf', BinaryRelevance(GaussianNB(
                    ))),
            ])
LogReg_pipeline = Pipeline([
               ('tfidf', TfidfVectorizer(stop_words=stopwords.words('russian'))),
               ('clf', BinaryRelevance(LogisticRegression(C=1500))),
           ])

sgd_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stopwords.words('russian'))),
                ('clf', BinaryRelevance(SGDClassifier(loss='hinge', penalty='l2',
                    alpha=1e-3, random_state=42, max_iter=5, tol=None))),
               ])

NB_pipeline.fit(X_train, train.loc[:,'salary':'court_threat'])
prediction = NB_pipeline.predict(X_test)
print(classification_report(test.loc[:,'salary':'court_threat'], prediction,target_names=labels))
print('accuracy_score: ', round(accuracy_score(test.loc[:,'salary':'court_threat'], prediction),4))
print('macro_avg f1-score: ', round(f1_score(test.loc[:,'salary':'court_threat'], prediction, average='macro'),4))
print('micro_avg f1-score: ', round(f1_score(test.loc[:,'salary':'court_threat'], prediction, average='micro'),4))


              precision    recall  f1-score   support

      salary       0.10      0.08      0.09        24
        boss       0.26      0.31      0.28        32
  atmosphere       0.41      0.49      0.45        35
 fire_threat       0.21      0.30      0.25        10
   work_cond       0.51      0.56      0.54        34
     neutral       0.88      0.62      0.73        24
court_threat       0.08      0.12      0.10         8

 avg / total       0.40      0.40      0.39       167

accuracy_score:  0.2358
macro_avg f1-score:  0.3481
micro_avg f1-score:  0.3862


C:\Users\vasil\Anaconda3\envs\dp\lib\site-packages\sklearn\metrics\classification.py:1428: UserWarning: labels size, 7, does not match size of target_names, 612
  .format(len(labels), len(target_names))


In [69]:
from skmultilearn.problem_transform import LabelPowerset

LogReg_pipeline = Pipeline([
               ('tfidf', TfidfVectorizer(stop_words=stopwords.words('russian'))),
               ('clf', LabelPowerset(LogisticRegression(penalty = 'l2', C = 1500,random_state = 0))),
           ])
NB_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stopwords.words('russian'))),
                ('clf', LabelPowerset(GaussianNB(
                    ))),
            ])
sgd_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stopwords.words('russian'))),
                ('clf', BinaryRelevance(SGDClassifier(loss='hinge', penalty='l2',
                    alpha=1e-3, random_state=42, max_iter=5, tol=None))),
               ])


LogReg_pipeline.fit(X_train, train.loc[:,'salary':'court_threat'])
prediction = LogReg_pipeline .predict(X_test)
print(classification_report(test.loc[:,'salary':'court_threat'], prediction,target_names=labels))
print('accuracy_score: ', round(accuracy_score(test.loc[:,'salary':'court_threat'], prediction),4))
print('macro_avg f1-score: ', round(f1_score(test.loc[:,'salary':'court_threat'], prediction, average='macro'),4))
print('micro_avg f1-score: ', round(f1_score(test.loc[:,'salary':'court_threat'], prediction, average='micro'),4))


              precision    recall  f1-score   support

      salary       0.92      0.46      0.61        24
        boss       0.76      0.50      0.60        32
  atmosphere       0.48      0.40      0.44        35
 fire_threat       0.44      0.40      0.42        10
   work_cond       0.67      0.41      0.51        34
     neutral       0.50      0.79      0.61        24
court_threat       0.33      0.12      0.18         8

 avg / total       0.63      0.47      0.52       167

accuracy_score:  0.3821
macro_avg f1-score:  0.4825
micro_avg f1-score:  0.5267


C:\Users\vasil\Anaconda3\envs\dp\lib\site-packages\sklearn\metrics\classification.py:1428: UserWarning: labels size, 7, does not match size of target_names, 612
  .format(len(labels), len(target_names))


In [54]:
from skmultilearn.adapt import MLkNN
from scipy.sparse import csr_matrix, lil_matrix

vectorizer = TfidfVectorizer()
vectorizer.fit(train.text)
vectorizer.fit(test.text)
X_train = vectorizer.transform(train.text)
X_test = vectorizer.transform(test.text)


x_train = lil_matrix(X_train).toarray()
y_train = lil_matrix(train.loc[:,'salary':'court_threat']).toarray()
x_test = lil_matrix(X_test).toarray()

"""
from sklearn.model_selection import GridSearchCV
parameters = {'k': [1,2,3,4,5]}
mlknn = MLkNN()
gs_clf_svm = GridSearchCV(mlknn, parameters, scoring='f1_micro')
gs_clf_svm = gs_clf_svm.fit(X_train,y_train)

print(gs_clf_svm.best_score_)
print(gs_clf_svm.best_params_)
"""
mlknn = MLkNN(2)
mlknn.fit(X_train,y_train)
prediction = mlknn.predict(x_test)
print(accuracy_score(test.loc[:,'salary':'court_threat'], prediction))
print(classification_report(test.loc[:,'salary':'court_threat'], prediction,target_names=labels))
print('macro_avg f1-score: ', round(f1_score(test.loc[:,'salary':'court_threat'], prediction, average='macro'),4))
print('micro_avg f1-score: ', round(f1_score(test.loc[:,'salary':'court_threat'], prediction, average='micro'),4))


0.2032520325203252
              precision    recall  f1-score   support

      salary       0.50      0.04      0.08        24
        boss       0.50      0.12      0.20        32
  atmosphere       0.67      0.06      0.11        35
 fire_threat       0.67      0.20      0.31        10
   work_cond       0.33      0.03      0.05        34
     neutral       0.20      0.92      0.33        24
court_threat       0.00      0.00      0.00         8

 avg / total       0.44      0.19      0.15       167

macro_avg f1-score:  0.1535
micro_avg f1-score:  0.2162


C:\Users\vasil\Anaconda3\envs\dp\lib\site-packages\sklearn\metrics\classification.py:1428: UserWarning: labels size, 7, does not match size of target_names, 612
  .format(len(labels), len(target_names))


In [49]:
SVC_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stopwords.words('russian'))),
                ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1)),
            ])

NB_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stopwords.words('russian'))),
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])
SVC_pipeline.fit(X_train,train.loc[:,'salary':'court_threat'])



prediction = SVC_pipeline.predict(X_test)
print(accuracy_score(test.loc[:,'salary':'court_threat'], prediction))

0.2764227642276423


In [33]:
model = Pipeline([('tfidf', TfidfVectorizer(stop_words=stopwords.words('russian'))),
 ('clf', OneVsRestClassifier(LinearSVC(class_weight='balanced')))])
#the class_weight="balanced" option tries to remove the biasedness of model towards majority sample

#paramater selection
    
from sklearn.model_selection import GridSearchCV
parameters = {'vectorizer__ngram_range': [(1, 1), (1, 2),(2,2)],
               'tfidf__use_idf': (True, False)}
gs_clf_svm = GridSearchCV(model, parameters, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(data['text'], data.loc[:,'salary':'court_threat'])

print(gs_clf_svm.best_score_)
print(gs_clf_svm.best_params_)

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
C:\Users\vasil\Anaconda3\envs\dp\lib\runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...\\dp\\lib\\site-packages\\ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
C:\Users\vasil\Anaconda3\envs

In [43]:
#LogReg_pipeline = Pipeline([
  #              ('tfidf', TfidfVectorizer(stop_words=stopwords.words('russian'))),
  #              ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=1)),
   #         ])
SVC_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stopwords.words('russian'))),
                ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1)),
            ])
NB_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stopwords.words('russian'))),
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])
"""
from sklearn.model_selection import GridSearchCV
parameters = {'vectorizer__ngram_range': [(1, 1), (1, 2),(2,2)],
               'tfidf__use_idf': (True, False)}

gs_log_reg = GridSearchCV(LogReg_pipeline, parameters, n_jobs=-1)
gs_log_reg = gs_log_reg.fit(data['text'], data.loc[:,'salary':'court_threat'])
print(gs_log_reg.best_score_)
print(gs_log_reg.best_params_)

for category in labels:
    print('Категория: {}'.format(category))
    LogReg_pipeline.fit(x_train, y_train[category])
    prediction = LogReg_pipeline.predict(x_test)
    print('Точность на тестовых данных: {}'.format(round(accuracy_score(y_test[category], prediction), 4)))
"""


"\nfrom sklearn.model_selection import GridSearchCV\nparameters = {'vectorizer__ngram_range': [(1, 1), (1, 2),(2,2)],\n               'tfidf__use_idf': (True, False)}\n\ngs_log_reg = GridSearchCV(LogReg_pipeline, parameters, n_jobs=-1)\ngs_log_reg = gs_log_reg.fit(data['text'], data.loc[:,'salary':'court_threat'])\nprint(gs_log_reg.best_score_)\nprint(gs_log_reg.best_params_)\n\nfor category in labels:\n    print('Категория: {}'.format(category))\n    LogReg_pipeline.fit(x_train, y_train[category])\n    prediction = LogReg_pipeline.predict(x_test)\n    print('Точность на тестовых данных: {}'.format(round(accuracy_score(y_test[category], prediction), 4)))\n"

In [44]:
for topic in labels:
    print('Категория: {}'.format(topic))
    NB_pipeline.fit(x_train, y_train[topic])
    prediction = NB_pipeline.predict(x_test)
    print('Точность на тестовых данных: {}'.format(
    round(accuracy_score(y_test[topic], prediction), 4)))

Категория: salary
Точность на тестовых данных: 0.8049
Категория: boss
Точность на тестовых данных: 0.7642
Категория: atmosphere
Точность на тестовых данных: 0.7154
Категория: fire_threat
Точность на тестовых данных: 0.9187
Категория: work_cond
Точность на тестовых данных: 0.7236
Категория: neutral
Точность на тестовых данных: 0.8049
Категория: court_threat
Точность на тестовых данных: 0.935
